# H3 Python API

In [1]:
from h3 import h3
import folium
from json import load

def visualize_hexagons(hexagons, color="red", folium_map=None):
    """
    hexagons is a list of hexcluster. Each hexcluster is a list of hexagons.
    eg. [[hex1, hex2], [hex3, hex4]]
    """
    polylines = []
    lat = []
    lng = []
    for hex in hexagons:
        polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
        # flatten polygons into loops.
        outlines = [loop for polygon in polygons for loop in polygon]
        polyline = [outline + [outline[0]] for outline in outlines][0]
        lat.extend(map(lambda v: v[0], polyline))
        lng.extend(map(lambda v: v[1], polyline))
        polylines.append(polyline)

    if folium_map is None:
        m = folium.Map(
            location=[sum(lat) / len(lat), sum(lng) / len(lng)],
            zoom_start=13,
            tiles="cartodbpositron",
        )
    else:
        m = folium_map
    for polyline in polylines:
        my_PolyLine = folium.PolyLine(locations=polyline, weight=8, color=color)
        m.add_child(my_PolyLine)
    return m


def visualize_polygon(polyline, color):
    polyline.append(polyline[0])
    lat = [p[0] for p in polyline]
    lng = [p[1] for p in polyline]
    m = folium.Map(
        location=[sum(lat) / len(lat), sum(lng) / len(lng)],
        zoom_start=13,
        tiles="cartodbpositron",
    )
    my_PolyLine = folium.PolyLine(locations=polyline, weight=8, color=color)
    m.add_child(my_PolyLine)
    return m

def style(feature):
        return {
            'fillColor': feature['properties']['color'],
            'color': feature['properties']['color'],
            'weight': 1
        }

with open("poland.country.json") as f:
    borders = load(f)

with open('aed_poland.geojson') as ps:
    pointsR = load(ps)

geoJson_border = borders["features"][0]["geometry"]
# swap lat and lng
geoJson_border["coordinates"][0] = [[p[1], p[0]] for p in geoJson_border["coordinates"][0]]

polyline = geoJson_border["coordinates"][0]
polyline.append(polyline[0])

lat = [p[0] for p in polyline]
lng = [p[1] for p in polyline]

m = folium.Map(
    location=[sum(lat) / len(lat), sum(lng) / len(lng)],
    zoom_start=5,
    tiles="cartodbpositron",
)

border_polyline = folium.PolyLine(locations=polyline, weight=8, color="green")
m.add_child(border_polyline)
h3_hexes = h3.polyfill(geoJson_border, 4)
print(h3_hexes)
hexagons = list(h3_hexes)
polylines = []
lat = []
lng = []
for hex in hexagons:
    polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
    # flatten polygons into loops.
    outlines = [loop for polygon in polygons for loop in polygon]
    polyline = [outline + [outline[0]] for outline in outlines][0]
    lat.extend(map(lambda v: v[0], polyline))
    lng.extend(map(lambda v: v[1], polyline))
    polylines.append(polyline)
for polyline in polylines:
    border_polyline = folium.PolyLine(locations=polyline, weight=8, style_function=style)
    m.add_child(border_polyline)
display(m)

{'841e2cbffffffff', '841f193ffffffff', '841e24bffffffff', '841e269ffffffff', '841e255ffffffff', '841f555ffffffff', '841e059ffffffff', '841e219ffffffff', '841f095ffffffff', '841f561ffffffff', '841e28bffffffff', '841f0b5ffffffff', '841f553ffffffff', '841e2c5ffffffff', '841f0ebffffffff', '841f0a9ffffffff', '841f089ffffffff', '841f565ffffffff', '841e2d3ffffffff', '841f54bffffffff', '841f425ffffffff', '841e291ffffffff', '841e2d1ffffffff', '841e283ffffffff', '841f511ffffffff', '841e23dffffffff', '841e2d9ffffffff', '841f0e3ffffffff', '841e2dbffffffff', '841f199ffffffff', '841e249ffffffff', '841e259ffffffff', '841e213ffffffff', '841e2a3ffffffff', '841f083ffffffff', '841f097ffffffff', '841e21bffffffff', '841e231ffffffff', '841e29bffffffff', '841f085ffffffff', '841e235ffffffff', '841f0b3ffffffff', '841f5e9ffffffff', '841e2c3ffffffff', '841f507ffffffff', '841e2a9ffffffff', '841f091ffffffff', '841f197ffffffff', '841e2e9ffffffff', '841e26bffffffff', '841e2b7ffffffff', '841f50dffffffff', '841e299fff

In [2]:
def aggregated_hex_points(points: list) -> dict:
    """
    points is a list of points. Each point is a list of lat and lng.
    eg. [[lat1, lng1], [lat2, lng2]]
    """
    hexagons = {}
    for point in points:
        hex = h3.geo_to_h3(point[0], point[1], 4)
        if hex not in hexagons:
            hexagons[hex] = 0
        hexagons[hex] += 1
    return hexagons

points_coordinates = [feature['geometry']['coordinates'] for feature in pointsR['features']]

h3_hexes = aggregated_hex_points(points_coordinates)

polylines = []
lat = []
lng = []
for hex in h3_hexes.keys():
    polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
    # flatten polygons into loops.
    outlines = [loop for polygon in polygons for loop in polygon]
    polyline = [outline + [outline[0]] for outline in outlines][0]
    lat.extend(map(lambda v: v[0], polyline))
    lng.extend(map(lambda v: v[1], polyline))
    # add color based on number of points
    # the value is the red rgb value
    color = f"#{h3_hexes[hex]:02}0000"
    print(color)
    # add color to the properties of the hexagon
    
    polylines.append(polyline)
for polyline in polylines:
    border_polyline = folium.PolyLine(locations=polyline, weight=8, style_function=style)
    m.add_child(border_polyline)
display(m)

#170000
#750000
#970000
#170000
#2640000
#830000
#1730000
#370000
#440000
#1310000
#320000
#1400000
#750000
#790000
#350000
#080000
#200000
#700000
#220000
#220000
#910000
#320000
#330000
#100000
#300000
#360000
#390000
#510000
#590000
#110000
#390000
#060000
#260000
#300000
#180000
#210000
#390000
#410000
#070000
#240000
#070000
#1870000
#310000
#160000
#130000
#120000
#210000
#120000
#040000
#460000
#070000
#190000
#210000
#200000
#090000
#300000
#640000
#170000
#120000
#190000
#1180000
#090000
#150000
#050000
#010000
#040000
#110000
#080000
#320000
#040000
#040000
#010000
#030000
#120000
#010000
#290000
#070000
#170000
#030000
#130000
#110000
#130000
#110000
#120000
#190000
#200000
#100000
#140000
#150000
#070000
#080000
#040000
#030000
#570000
#210000
#340000
#040000
#060000
#570000
#090000
#070000
#070000
#020000
#210000
#160000
#060000
#330000
#040000
#100000
#060000
#440000
#040000
#060000
#040000
#060000
#040000
#180000
#380000
#260000
#280000
#110000
#390000
#070000
#120000
#1